# Pipeline NLP - Analisi e Modellazione Testo

Questo notebook contiene tutte le procedure standard per un progetto di Natural Language Processing:
- Caricamento e esplorazione dati
- Pulizia e preprocessing del testo
- Analisi esplorativa
- Feature extraction
- Modellazione
- Valutazione

## 1. Import Librerie

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import re
from tqdm import tqdm
tqdm.pandas()

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Librerie di base caricate con successo!")

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

print("NLTK configurato!")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score

print("Scikit-learn pronto!")

## 2. Configurazione Percorsi

In [ ]:
BASE_DIR = Path('..')
DATA_DIR = BASE_DIR / 'data'
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
CLEANED_DATA_DIR = DATA_DIR / 'cleaned'
MODELS_DIR = BASE_DIR / 'models'

for directory in [RAW_DATA_DIR, PROCESSED_DATA_DIR, CLEANED_DATA_DIR, MODELS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print(f"Cartelle configurate:")
print(f"  - Dati raw: {RAW_DATA_DIR}")
print(f"  - Dati processati: {PROCESSED_DATA_DIR}")
print(f"  - Dati puliti: {CLEANED_DATA_DIR}")
print(f"  - Modelli: {MODELS_DIR}")

## 3. Caricamento Dati

Modifica questa cella in base al formato dei tuoi dati (CSV, JSON, Excel, TXT, ecc.)

In [ ]:
# df = pd.read_csv(RAW_DATA_DIR / 'nome_file.csv')
# df = pd.read_excel(RAW_DATA_DIR / 'nome_file.xlsx')
# df = pd.read_json(RAW_DATA_DIR / 'nome_file.json')

# Esempio con dati di test
df = pd.DataFrame({
    'text': [
        'Questo è un esempio di testo da analizzare.',
        'Il Natural Language Processing è molto utile!',
        'Python è ottimo per il machine learning.',
    ],
    'label': [0, 1, 1]
})

print(f"Dataset caricato: {len(df)} righe")
df.head()

## 4. Esplorazione Dati

In [ ]:
print("=== INFO DATASET ===")
print(f"Numero di righe: {len(df)}")
print(f"Numero di colonne: {len(df.columns)}")
print(f"\nColonne: {list(df.columns)}")
print(f"\nTipi di dati:")
print(df.dtypes)
print(f"\nValori mancanti:")
print(df.isnull().sum())
print(f"\nStatistiche:")
df.describe()

In [ ]:
# Modifica 'text' con il nome della colonna che contiene il testo
text_column = 'text'

if text_column in df.columns:
    df['text_length'] = df[text_column].astype(str).str.len()
    df['word_count'] = df[text_column].astype(str).str.split().str.len()
    
    print(f"\n=== STATISTICHE TESTO ===")
    print(f"Lunghezza media: {df['text_length'].mean():.2f} caratteri")
    print(f"Lunghezza minima: {df['text_length'].min()} caratteri")
    print(f"Lunghezza massima: {df['text_length'].max()} caratteri")
    print(f"\nNumero medio di parole: {df['word_count'].mean():.2f}")
    print(f"Numero minimo di parole: {df['word_count'].min()}")
    print(f"Numero massimo di parole: {df['word_count'].max()}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].hist(df['text_length'], bins=50, edgecolor='black')
axes[0].set_title('Distribuzione Lunghezza Testi')
axes[0].set_xlabel('Numero di caratteri')
axes[0].set_ylabel('Frequenza')

axes[1].hist(df['word_count'], bins=50, edgecolor='black', color='orange')
axes[1].set_title('Distribuzione Numero Parole')
axes[1].set_xlabel('Numero di parole')
axes[1].set_ylabel('Frequenza')

plt.tight_layout()
plt.show()

In [ ]:
# Se hai una colonna con le etichette/classi
label_column = 'label'  # Modifica con il nome della tua colonna

if label_column in df.columns:
    print(f"\n=== DISTRIBUZIONE CLASSI ===")
    print(df[label_column].value_counts())
    print(f"\nProporzioni:")
    print(df[label_column].value_counts(normalize=True))
    
    plt.figure(figsize=(10, 6))
    df[label_column].value_counts().plot(kind='bar')
    plt.title('Distribuzione delle Classi')
    plt.xlabel('Classe')
    plt.ylabel('Frequenza')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 5. Pulizia del Testo

Funzioni per pulire e normalizzare il testo

In [ ]:
def clean_text(text):
    """
    Pulisce il testo:
    - Rimuove URL
    - Rimuove email
    - Rimuove menzioni (@username)
    - Rimuove hashtag (#tag)
    - Rimuove caratteri speciali e numeri
    - Converte in minuscolo
    - Rimuove spazi multipli
    """
    if pd.isna(text):
        return ''
    
    text = str(text)
    
    # Rimuovi URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Rimuovi email
    text = re.sub(r'\S+@\S+', '', text)
    
    # Rimuovi menzioni e hashtag
    text = re.sub(r'@\w+|#\w+', '', text)
    
    # Rimuovi numeri
    text = re.sub(r'\d+', '', text)
    
    # Rimuovi punteggiatura e caratteri speciali
    text = re.sub(r'[^\w\s]', '', text)
    
    # Converti in minuscolo
    text = text.lower()
    
    # Rimuovi spazi multipli
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

print("Funzione clean_text() definita")

# Test
sample_text = "Ciao! Visita https://example.com e contattami a test@email.com #NLP @user123"
print(f"\nTesto originale: {sample_text}")
print(f"Testo pulito: {clean_text(sample_text)}")

In [ ]:
# Applica la pulizia al dataset
print("Pulizia del testo in corso...")
df['cleaned_text'] = df[text_column].progress_apply(clean_text)

# Rimuovi righe con testo vuoto dopo la pulizia
df_cleaned = df[df['cleaned_text'].str.len() > 0].copy()

print(f"\nRighe originali: {len(df)}")
print(f"Righe dopo pulizia: {len(df_cleaned)}")
print(f"Righe rimosse: {len(df) - len(df_cleaned)}")

df_cleaned[['text', 'cleaned_text']].head()

## 6. Preprocessing Avanzato

Tokenizzazione, rimozione stopwords, stemming/lemmatization

In [ ]:
# Configura le stopwords (puoi scegliere la lingua)
# Lingue disponibili: 'italian', 'english', 'spanish', 'french', 'german', ecc.
language = 'italian'  # Modifica in base alla tua lingua

try:
    stop_words = set(stopwords.words(language))
    print(f"Stopwords caricate per la lingua: {language}")
    print(f"Numero di stopwords: {len(stop_words)}")
    print(f"\nEsempi di stopwords: {list(stop_words)[:10]}")
except:
    print(f"Lingua '{language}' non disponibile, uso l'inglese")
    stop_words = set(stopwords.words('english'))

In [ ]:
def preprocess_text(text, remove_stopwords=True, use_lemmatization=True):
    """
    Preprocessing del testo:
    - Tokenizzazione
    - Rimozione stopwords (opzionale)
    - Lemmatization o Stemming (opzionale)
    """
    if pd.isna(text) or text == '':
        return []
    
    # Tokenizzazione
    tokens = word_tokenize(str(text))
    
    # Rimuovi stopwords
    if remove_stopwords:
        tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # Lemmatization o Stemming
    if use_lemmatization:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
    else:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
    
    return tokens

print("Funzione preprocess_text() definita")

# Test
sample = "Gli studenti stanno studiando per gli esami"
print(f"\nTesto: {sample}")
print(f"Tokens: {preprocess_text(sample)}")

In [ ]:
# Applica il preprocessing
print("Preprocessing in corso...")
df_cleaned['tokens'] = df_cleaned['cleaned_text'].progress_apply(
    lambda x: preprocess_text(x, remove_stopwords=True, use_lemmatization=True)
)

# Crea una versione del testo con i token uniti
df_cleaned['processed_text'] = df_cleaned['tokens'].apply(lambda x: ' '.join(x))

# Rimuovi righe senza token
df_cleaned = df_cleaned[df_cleaned['processed_text'].str.len() > 0].copy()

print(f"\nRighe dopo preprocessing: {len(df_cleaned)}")
df_cleaned[['cleaned_text', 'processed_text']].head()

## 7. Analisi Esplorativa Post-Processing

In [ ]:
from collections import Counter

# Conta le parole più frequenti
all_words = [word for tokens in df_cleaned['tokens'] for word in tokens]
word_freq = Counter(all_words)

print(f"Numero totale di parole uniche: {len(word_freq)}")
print(f"\nTop 20 parole più frequenti:")
for word, count in word_freq.most_common(20):
    print(f"  {word}: {count}")

In [ ]:
# Visualizza le parole più frequenti
top_words = dict(word_freq.most_common(20))

plt.figure(figsize=(12, 6))
plt.bar(top_words.keys(), top_words.values())
plt.title('Top 20 Parole Più Frequenti')
plt.xlabel('Parola')
plt.ylabel('Frequenza')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# WordCloud (nuvola di parole)
try:
    from wordcloud import WordCloud
    
    all_text = ' '.join(df_cleaned['processed_text'])
    
    wordcloud = WordCloud(width=800, height=400, 
                         background_color='white',
                         max_words=100).generate(all_text)
    
    plt.figure(figsize=(15, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('WordCloud del Corpus', fontsize=20)
    plt.tight_layout()
    plt.show()
except ImportError:
    print("WordCloud non disponibile. Installa con: pip install wordcloud")

## 8. Salvataggio Dati Processati

In [ ]:
# Salva il dataset pulito
output_file = CLEANED_DATA_DIR / 'data_cleaned.csv'
df_cleaned.to_csv(output_file, index=False)
print(f"Dataset pulito salvato in: {output_file}")

# Salva anche in formato pickle per preservare i tipi di dato
output_pickle = CLEANED_DATA_DIR / 'data_cleaned.pkl'
df_cleaned.to_pickle(output_pickle)
print(f"Dataset salvato anche in: {output_pickle}")

## 9. Feature Extraction

Trasforma il testo in features numeriche per il machine learning

### 9.1 Bag of Words (BoW)

In [ ]:
# CountVectorizer - Bag of Words
count_vectorizer = CountVectorizer(max_features=100, min_df=2)
bow_features = count_vectorizer.fit_transform(df_cleaned['processed_text'])

print(f"Shape matrice BoW: {bow_features.shape}")
print(f"Numero di features: {len(count_vectorizer.get_feature_names_out())}")
print(f"\nEsempio features: {list(count_vectorizer.get_feature_names_out()[:20])}")

### 9.2 TF-IDF

In [ ]:
# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=100, min_df=2)
tfidf_features = tfidf_vectorizer.fit_transform(df_cleaned['processed_text'])

print(f"Shape matrice TF-IDF: {tfidf_features.shape}")
print(f"Numero di features: {len(tfidf_vectorizer.get_feature_names_out())}")
print(f"\nEsempio features: {list(tfidf_vectorizer.get_feature_names_out()[:20])}")

## 10. Preparazione per Modellazione

Split train/test per supervisionati

In [ ]:
# Se hai una colonna target per classificazione/regressione
if label_column in df_cleaned.columns:
    X = tfidf_features  # o bow_features
    y = df_cleaned[label_column]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\n=== SPLIT DATASET ===")
    print(f"Training set: {X_train.shape[0]} campioni")
    print(f"Test set: {X_test.shape[0]} campioni")
    print(f"\nDistribuzione classi nel training set:")
    print(y_train.value_counts())
    print(f"\nDistribuzione classi nel test set:")
    print(y_test.value_counts())
else:
    print("Nessuna colonna target trovata - preparazione per unsupervised learning")
    X = tfidf_features

## 11. Modellazione Base

Esempi di modelli semplici da provare

In [ ]:
# Esempio: Naive Bayes
from sklearn.naive_bayes import MultinomialNB

if label_column in df_cleaned.columns:
    print("Training Naive Bayes...")
    nb_model = MultinomialNB()
    nb_model.fit(X_train, y_train)
    
    y_pred = nb_model.predict(X_test)
    
    print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"\nClassification Report:\n{classification_report(y_test, y_pred)}")
else:
    print("Modello supervisionato non applicabile - nessuna colonna target")

In [ ]:
# Esempio: Logistic Regression
from sklearn.linear_model import LogisticRegression

if label_column in df_cleaned.columns:
    print("Training Logistic Regression...")
    lr_model = LogisticRegression(max_iter=1000, random_state=42)
    lr_model.fit(X_train, y_train)
    
    y_pred = lr_model.predict(X_test)
    
    print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"\nClassification Report:\n{classification_report(y_test, y_pred)}")

In [ ]:
# Confusion Matrix
if label_column in df_cleaned.columns:
    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

## 12. Salvataggio Modello

In [ ]:
import pickle

# Salva il modello e il vectorizer
if label_column in df_cleaned.columns:
    with open(MODELS_DIR / 'model.pkl', 'wb') as f:
        pickle.dump(lr_model, f)
    
    with open(MODELS_DIR / 'vectorizer.pkl', 'wb') as f:
        pickle.dump(tfidf_vectorizer, f)
    
    print(f"Modello salvato in: {MODELS_DIR / 'model.pkl'}")
    print(f"Vectorizer salvato in: {MODELS_DIR / 'vectorizer.pkl'}")

## 13. Note e Prossimi Passi

### Cosa puoi fare ora:

1. **Carica i tuoi dati** nella sezione 3
2. **Adatta le funzioni di pulizia** alle tue esigenze (sezione 5)
3. **Scegli la lingua** per le stopwords (sezione 6)
4. **Prova diversi modelli**:
   - Naive Bayes
   - Logistic Regression
   - SVM
   - Random Forest
   - XGBoost
   - Deep Learning (LSTM, Transformers)

5. **Tecniche avanzate da esplorare**:
   - Word Embeddings (Word2Vec, GloVe, FastText)
   - Transfer Learning (BERT, GPT)
   - Topic Modeling (LDA)
   - Named Entity Recognition (NER)
   - Sentiment Analysis
   - Text Generation

6. **Ottimizzazione**:
   - Hyperparameter tuning (GridSearch, RandomSearch)
   - Cross-validation
   - Feature engineering avanzato
   - Ensemble methods